### Notes

Master DB has:
 - rep_test1.rep imported with default importer
 - gpx_1_0.gpx imported with platform NELSON created with sensor GPS
 
Slave DB has:
 - gpx_1_1.gpx imported with platform NELSON created with sensor GPS

In [25]:
from pepys_import.core.store.data_store import DataStore
from sqlalchemy.orm.session import make_transient
from sqlalchemy.orm import undefer

In [2]:
master_store = DataStore("", "", "", 0, db_name="master.sqlite", db_type="sqlite")
slave_store = DataStore("", "", "", 0, db_name="slave.sqlite", db_type="sqlite")

   @@@@ @@@@@        ______                      _                            _   
 @@@@@ @@..@@@@      | ___ \                    (_)                          | |  
  @@@     @@@@@@     | |_/ /__ _ __  _   _ ___   _ _ __ ___  _ __   ___  _ __| |_ 
  @@@     @@@@@@@@   |  __/ _ \ '_ \| | | / __| | | '_ ` _ \| '_ \ / _ \| '__| __|
 @@@@.     @@@@@@@   | | |  __/ |_) | |_| \__ \ | | | | | | | |_) | (_) | |  | |_ 
 @@ #@@ @   ...@@@@@ \_|  \___| .__/ \__, |___/ |_|_| |_| |_| .__/ \___/|_|   \__|
 @@   &        @@@@@            | |     __/ | ______          | |                   
Software Version :  0.0.10 


Database Type :  sqlite
Database Name :  master.sqlite
Database Host :  
-------------------------------------------------------------
   @@@@ @@@@@        ______                      _                            _   
 @@@@@ @@..@@@@      | ___ \                    (_)                          | |  
  @@@     @@@@@@     | |_/ /__ _ __  _   _ ___   _ _ __ ___  _ __   ___  _ __| |_ 
  @

In [22]:
with master_store.session_scope():
    res = master_store.session.query(master_store.db_classes.Sensor).all()
    print(res)
    print(len(res))

[<pepys_import.core.store.sqlite_db.Sensor object at 0x10fc26e10>, <pepys_import.core.store.sqlite_db.Sensor object at 0x10fc26e90>, <pepys_import.core.store.sqlite_db.Sensor object at 0x10fc39490>, <pepys_import.core.store.sqlite_db.Sensor object at 0x10fc39810>, <pepys_import.core.store.sqlite_db.Sensor object at 0x10fc39b90>]
5


In [27]:
with slave_store.session_scope():
    res = slave_store.session.query(slave_store.db_classes.SensorType).options(undefer("*")).all()
    print(res)
    print(len(res))
    
    slave_store.session.expunge_all()
    with master_store.session_scope():
        for r in res[0:1]:
            make_transient(r)
            master_store.session.add(r)
        print(master_store.session.new)
        print(r in master_store.session)
        master_store.session.flush()

[<pepys_import.core.store.sqlite_db.SensorType object at 0x110a90490>, <pepys_import.core.store.sqlite_db.SensorType object at 0x110a90610>, <pepys_import.core.store.sqlite_db.SensorType object at 0x110a90790>]
3
IdentitySet([<pepys_import.core.store.sqlite_db.SensorType object at 0x110a90490>])
True


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: SensorTypes.name
[SQL: INSERT INTO "SensorTypes" (sensor_type_id, name, created_date) VALUES (?, ?, ?)]
[parameters: ('79f43bb0ebe0481991ca1e78bf27d459', 'GPS', '2020-05-20 09:13:11.979908')]
(Background on this error at: http://sqlalche.me/e/gkpj)

In [12]:
s = res[0]

In [ ]:
master_store.session.commit()